## Faltung ##

Dieses Notebook demonstriert die Faltung interaktiv

In [ ]:
%matplotlib inline
# "inline" renders using inline png graphics
from __future__ import print_function, division
import sys
import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
from matplotlib import animation
from IPython.display import HTML
from IPython import __version__ as ipy_version

sys.path.append('../..')
from dsp_fpga_lib import versions
versions()
print("IPython:", ipy_version)

plt.rcParams['figure.figsize'] = (5,3)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
#plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

In [ ]:
%%capture
fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True)
l1, = ax[0].plot([],[])
ml, sl, bl = ax[1].stem(n, x, '-.')

ax[0].grid(True)
ax[0].set_ylabel(r'$y[n] \rightarrow$')
ax[0].set_title(r'Faltung $y[n] = x[n] \star h[n]$')
ax[0].set_xlim(( 0, L))
ax[0].set_ylim((-2, 2));
ax[1].set_title(r'Faltung $bla$')
ax[1].set_xlabel(r'$n \rightarrow$');

In [ ]:
# define signal and impulse response here
h = [0.25, 0.5, 0.25] # impulse response
x = [1, 1, 1, 1, 1] # input signal 
N = 0 # max. number of data points
L = max(N, len(h) + len(x))

n = np.arange(L)
h = np.append(h, np.zeros(L - len(h))) # fill up with zeros
x = np.append(x, np.zeros(L - len(x)))

y = np.convolve(x, h)[:L] 



In [ ]:
def animate(i):
    y = np.roll(x, i)
    l1.set_data(n, y)
    ax[1].cla()
    ml, sl, l = ax[1].stem(n, y)
    ax[1].set_ylim((-1.5, 1.5))

ani = animation.FuncAnimation(fig, animate, frames=len(t))
ani

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
#fig = plt.figure(1) 
#ax = fig.add_subplot(111)
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 2, 100)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    if i%10 == 0: print('*', end='') # print a progress bar
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20)#, blit=True) # 100 frames with 20 ms intervals

# HTML(anim.to_html5_video())
anim